# Ray hyperparameter optimization Notebook

In [42]:
# Config
# sbatch_cmd_options = \
# "--time=00:30:00 \
# -q debug \
# -A dasrepo \
# --image=nersc/pytorch:ngc-22.05-v1 \
# --nodes=2"
sbatch_cmd_options = \
"--time=00:30:00 \
-q debug \
-A dasrepo \
--nodes=2"

In [43]:
# Run cell to setup RAY Cluster
!source scripts/submit_ray_cluster.sh "$sbatch_cmd_options"

<> Submiting Ray cluster job
Submitted batch job 66136249


In [44]:
#While loop pull contents
import os 
ray_scratch_file = os.path.join(os.getenv('SCRATCH'), 'ray_cluster', 'head_node_address')

print(ray_scratch_file)

/global/cscratch1/sd/asnaylor/ray_cluster/head_node_address


In [45]:
!ls /global/cscratch1/sd/asnaylor/ray_cluster/

head_node_address


In [46]:
#srun parts
!cat $ray_scratch_file

nid00911:6379


In [57]:
!sqs

JOBID            ST USER      NAME          NODES TIME_LIMIT       TIME  SUBMIT_TIME          QOS             START_TIME           FEATURES       NODELIST(REASON
66136249         CG asnaylor  sbatch_submi  2          30:00       2:45  2023-01-19T18:30:17  debug_hsw       2023-01-19T18:30:21  haswell        nid00[911-912] 


In [56]:
!scancel -u $USER

In [ ]:
# !cat slurm-*.out
!cat slurm-66136249.out

-----

In [52]:
import ray
import math

def convert_size(size_bytes):
    if size_bytes == 0:
        return "0B"
    size_name = ("B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB", "YB")
    i = int(math.floor(math.log(size_bytes, 1024)))
    p = math.pow(1024, i)
    s = round(size_bytes / p, 2)
    return "%s %s" % (s, size_name[i])


#get address from file
with open(ray_scratch_file) as f:
    RAY_NODE_ADDRESS = f.read().strip('\n')

print(RAY_NODE_ADDRESS)

nid00911:6379


In [55]:
ray.init(address=RAY_NODE_ADDRESS)
# ray.init(address='ray://nid00911:10001')

2023-01-19 18:32:38,615	INFO worker.py:1333 -- Connecting to existing Ray cluster at address: nid00911:6379...
2023-01-19 18:32:43,630	WARNING utils.py:1333 -- Unable to connect to GCS at nid00911:6379. Check that (1) Ray GCS with matching version started successfully at the specified address, and (2) there is no firewall setting preventing access.
2023-01-19 18:32:50,645	WARNING utils.py:1333 -- Unable to connect to GCS at nid00911:6379. Check that (1) Ray GCS with matching version started successfully at the specified address, and (2) there is no firewall setting preventing access.
2023-01-19 18:32:57,661	WARNING utils.py:1333 -- Unable to connect to GCS at nid00911:6379. Check that (1) Ray GCS with matching version started successfully at the specified address, and (2) there is no firewall setting preventing access.


KeyboardInterrupt: 

In [ ]:
nodes = ray.nodes()
print(nodes)

In [ ]:
node_resources = ray.cluster_resources()
print("total of {} CPUs and {} RAM".format(node_resources['CPU'], convert_size(node_resources['memory'])))
